# Chapter 1: Python GraphBLAS

Using the python-graphblas library for sparse linear algebra on graphs.

In [ ]:
import graphblas as gb
from graphblas import Matrix, Vector, binary, semiring
import networkx as nx
import matplotlib.pyplot as plt

## Creating Sparse Matrices

In [ ]:
# Create adjacency matrix from edge list (COO format)
rows = [0, 0, 1, 2, 3]
cols = [1, 2, 2, 3, 1]
vals = [1, 1, 1, 1, 1]

A = Matrix.from_coo(rows, cols, vals, nrows=4, ncols=4, dtype=int)
print(A)

In [ ]:
# Visualize the graph
G = nx.DiGraph()
G.add_edges_from(zip(rows, cols))
nx.draw(G, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=16, arrows=True)
plt.show()

## Creating Sparse Vectors

In [ ]:
# Create a vector marking node 0 as the starting point
v = Vector.from_coo([0], [1], size=4, dtype=int)
print("Starting vector (source at node 0):")
print(v)

## Simple BFS with Matrix-Vector Multiply

In [ ]:
# One hop: v × A (transpose) using mxv
frontier = v.mxv(A.T, semiring.any_pair).new()
print("After 1 hop:")
print(frontier)

In [ ]:
# Two hops
frontier2 = frontier.mxv(A.T, semiring.any_pair).new()
print("After 2 hops:")
print(frontier2)

In [ ]:
# BFS loop - expand until no new nodes
frontier = Vector.from_coo([0], [1], size=4, dtype=int)
visited = frontier.dup()

level = 0
while frontier.nvals > 0:
    print(f"Level {level}: nodes {list(frontier.to_coo()[0])}")
    frontier = frontier.mxv(A.T, semiring.any_pair).new()
    # Remove already visited
    frontier(mask=visited, replace=True) << frontier
    frontier(mask=~visited.S) << frontier
    visited(binary.any) << frontier
    level += 1

print(f"\nAll visited nodes: {list(visited.to_coo()[0])}")